In [1]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

In [2]:
import os
os.system('pip install netCDF4')
import netCDF4
#from mpl_toolkits.basemap import Basemap
#для создания карт: folium, geopandas, branca

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 KB 5.7 MB/s eta 0:00:00


In [3]:
SCAN_NEIGHBOURS_DISTANCE = 2  # +/- 2 cells

In [4]:
def read_netcdf(filename):
    file = netCDF4.Dataset(filename)
    data = file.variables[:]   # ['data'][:]  # keys ['time', 'zlev', 'lat', 'lon', 'u', 'v']
    file.close()

folder = '../www.ncei.noaa.gov/data/blended-global-sea-surface-wind-products/access/winds/6-hourly/'
file = netCDF4.Dataset(folder + '1995/uv19950115.nc')

wind_files = {}
for root, subdirs, files in os.walk('../www.ncei.noaa.gov'):  #'./Blended_Sea_Winds_dataset_6hourly'):
    for file in files:
        if file.endswith('.nc'):
            #print(root, subdirs, file)
            wind_files[file] = root + '/' + file

print(len(wind_files), list(wind_files.keys())[:5], list(wind_files.keys())[-5:])
print(list(wind_files.keys())[4], wind_files['uv19950105.nc'])
#print(read_netcdf(wind_files['uv19950105.nc']))

2193 ['uv19950101.nc', 'uv19950102.nc', 'uv19950103.nc', 'uv19950104.nc', 'uv19950105.nc'] ['uv20051227.nc', 'uv20051228.nc', 'uv20051229.nc', 'uv20051230.nc', 'uv20051231.nc']
uv19950105.nc ../www.ncei.noaa.gov/data/blended-global-sea-surface-wind-products/access/winds/6-hourly/1995/uv19950105.nc


In [13]:
fn = wind_files['uv20051227.nc']
file = netCDF4.Dataset(fn)

In [14]:
dir(file)[40:]

['_ncstring_attrs__',
 '_redef',
 'close',
 'cmptypes',
 'createCompoundType',
 'createDimension',
 'createEnumType',
 'createGroup',
 'createVLType',
 'createVariable',
 'data_model',
 'delncattr',
 'dimensions',
 'disk_format',
 'enumtypes',
 'file_format',
 'filepath',
 'fromcdl',
 'get_variables_by_attributes',
 'getncattr',
 'groups',
 'institution',
 'isopen',
 'keepweakref',
 'name',
 'ncattrs',
 'parent',
 'path',
 'references',
 'renameAttribute',
 'renameDimension',
 'renameGroup',
 'renameVariable',
 'set_always_mask',
 'set_auto_chartostring',
 'set_auto_mask',
 'set_auto_maskandscale',
 'set_auto_scale',
 'set_fill_off',
 'set_fill_on',
 'set_ncstring_attrs',
 'setncattr',
 'setncattr_string',
 'setncatts',
 'source',
 'summary',
 'sync',
 'title',
 'tocdl',
 'variables',
 'vltypes']

In [16]:
file.dimensions

{'zlev': <class 'netCDF4._netCDF4.Dimension'>: name = 'zlev', size = 1,
 'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 4,
 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 719,
 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440}

In [34]:
print(file.variables['time'][:])
file.variables['lon'][0:8], file.variables['zlev'][-2:]

[245328 245334 245340 245346]


(masked_array(data=[10.],
              mask=False,
        fill_value=1e+20,
             dtype=float32),
 masked_array(data=[10.],
              mask=False,
        fill_value=1e+20,
             dtype=float32))

In [18]:
print(file.variables.keys())
print(file.title)
print(file.summary)
for i in file.variables:
    print(i, file.variables[i][:3])

dict_keys(['time', 'zlev', 'lat', 'lon', 'u', 'v'])
NOAA/NCDC Blended 6-hourly 0.25-degree Sea Surface Winds
Gridded and blended sea surface vector winds from multiple satellites with direction from NCEP Reanalysis-2; Global ocean coverage with a 0.25-degree resolution; The whole datasets covers from July 1987 to present, 6-hourly resolution in this dataset; daily and monthly are also available in other directories; See http://www.ncdc.noaa.gov/oa/rsad/blendedseawinds.html and links within for details. Include (u,v) means and scalar mean speed w for comparison
time [245328 245334 245340]
zlev [10.]
lat [-89.75 -89.5  -89.25]
lon [0.   0.25 0.5 ]
u [[[[-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]
   ...
   [-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]]]


 [[[-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]
   ...
   [-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]
   [-- -- -- ... -- -- --]]]


In [17]:
# If you want to read data from a variable that spans multiple netCDF files, you can 
# use the MFDataset class to read the data as if it were contained in a single file. 

In [38]:
for i in file.variables:
    print(i, len(file.variables[i]))
print(file.variables['v'][0][0][300][300])#[245328,10,0,0]
print(file.variables['v'][1][0][300][300])
print(file.variables['v'][2][0][300][300]) # current shape = (4, 1, 719, 1440)

time 4
zlev 1
lat 719
lon 1440
u 4
v 4


IndexError: index 1 is out of bounds for axis 0 with size 1

In [30]:
t = file.variables['time']
netCDF4.num2date(t[:],t.units)

masked_array(data=[cftime.DatetimeGregorian(2005, 12, 27, 0, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2005, 12, 27, 6, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2005, 12, 27, 12, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2005, 12, 27, 18, 0, 0, 0, has_year_zero=False)],
             mask=False,
       fill_value='?',
            dtype=object)

In [35]:
#dir(file.variables['u'][0,0,0,0])

In [36]:
print(file.variables['v'])

<class 'netCDF4._netCDF4.Variable'>
float32 v(time, zlev, lat, lon)
    long_name: Sea Surface Wind: y-component
    units: m/s
    _FillValue: -9999.0
unlimited dimensions: 
current shape = (4, 1, 719, 1440)
filling on


In [46]:
file_c = netCDF4.Dataset('../oscar/world_oscar_vel_5d1995.nc')
print(file_c.variables['v'])
# float32 v(time, depth, latitude, longitude)
# current shape = (72, 1, 140, 360)
print(file_c.variables['v'][1,0,99,181])

<class 'netCDF4._netCDF4.Variable'>
float32 v(time, depth, latitude, longitude)
    units: meter/sec
    long_name: Meridional Velocity
    missing_value: nan
unlimited dimensions: 
current shape = (72, 1, 140, 360)
filling on, default _FillValue of 9.969209968386869e+36 used
-0.031576388


In [45]:
print(file_c.variables.keys())
for i in file_c.variables:
    print(i, file_c.variables[i])#[:1])

dict_keys(['latitude', 'longitude', 'time', 'date', 'depth', 'mask', 'u', 'v', 'uf', 'vf'])
latitude <class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    units: degrees-north
    long_name: Latitude
unlimited dimensions: 
current shape = (140,)
filling on, default _FillValue of 9.969209968386869e+36 used
longitude <class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    units: degrees-east
    long_name: Longitude
unlimited dimensions: 
current shape = (360,)
filling on, default _FillValue of 9.969209968386869e+36 used
time <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: day since 1992-10-05 00:00:00
    long_name: Time
unlimited dimensions: 
current shape = (72,)
filling on, default _FillValue of -2147483647 used
date <class 'netCDF4._netCDF4.Variable'>
int32 date(time)
    long_name: Date in integer format
unlimited dimensions: 
current shape = (72,)
filling on, default _FillValue of -2147483647 used
depth <class 'netCDF4._netCDF4.Variable'>


In [49]:
file_c = netCDF4.Dataset('../seanoe/ASAR_Product_NC_2003/ENVISAT_ASAR_Wave_20030101_000404_20030101_000649_00012_00316.nc')
print(file_c.variables['latitude'])
# float32 v(time, depth, latitude, longitude)
# current shape = (72, 1, 140, 360)
print(file_c.variables['v'][1,0,99,181])

<class 'netCDF4._netCDF4.Variable'>
float32 latitude(time)
    long_name: Latitude
    standard_name: latitude
    units: degrees_north
    valid_max: 90.0
    valid_min: -90.0
    comment: Positive latitude is North latitude, negative latitude is South latitude
unlimited dimensions: 
current shape = (12,)
filling on, default _FillValue of 9.969209968386869e+36 used


KeyError: 'v'

In [51]:
print(file_c.variables.keys())
for i in file_c.variables:
    print(i, file_c.variables[i][:2], file_c.variables[i][-2:])

dict_keys(['latitude', 'longitude', 'time', 'heading', 'inci_angle', 'land_flag', 'homogeneity', 'normalized_variance', 'rejection_flag', 'qc_flag', 'swh', 'mwp', 'swh_cali', 'mwp_cali'])
latitude [54.875015 55.748432] [63.574326  0.      ]
longitude [-38.59197 -38.96503] [-42.962505   0.      ]
time [94694644 94694659] [94694794 94694809]
heading [-16.88436 -17.15604] [-20.56627   0.     ]
inci_angle [22.767782 22.731747] [22.782444 22.75276 ]
land_flag [0 0] [1 0]
homogeneity [0.992 0.988] [-- --]
normalized_variance [1.1669663 1.1735283] [1.0331609 1.0004587]
rejection_flag [0 0] [2 1]
qc_flag [0 0] [-- --]
swh [3.353 3.476] [-- --]
mwp [7.404 7.379] [-- --]
swh_cali [3.42  3.559] [-- --]
mwp_cali [7.502 7.469] [-- --]


In [52]:
file_c = netCDF4.Dataset('../seanoe/ASAR_Product_NC_2003/ENVISAT_ASAR_Wave_20030730_212537_20030730_214551_00018_00329.nc')
print(file_c.variables.keys())
for i in file_c.variables:
    print(i, file_c.variables[i][:2], file_c.variables[i][-2:])

dict_keys(['latitude', 'longitude', 'time', 'heading', 'inci_angle', 'land_flag', 'homogeneity', 'normalized_variance', 'rejection_flag', 'qc_flag', 'swh', 'mwp', 'swh_cali', 'mwp_cali'])
latitude [-38.56931  -37.694366] [32.27441 33.15805]
longitude [19.008629 18.699593] [1.363645 1.141778]
time [112915537 112915552] [112916736 112916751]
heading [-14.024046 -13.925429] [-13.31665  -13.382732]
inci_angle [23.083258 23.153648] [22.751688 22.72313 ]
land_flag [0 0] [1 1]
homogeneity [0.986 1.002] [-- --]
normalized_variance [1.308798 1.344704] [1.1086711 1.846629 ]
rejection_flag [0 0] [2 2]
qc_flag [0 0] [-- --]
swh [5.324 6.878] [-- --]
mwp [8.005 8.625] [-- --]
swh_cali [5.666 7.438] [-- --]
mwp_cali [8.263 9.049] [-- --]


In [55]:

print(file_c.title)
print(file_c.summary)
dir(file_c)

Wave Parameter Product Derived from ESA ENVISAT/ASAR L1B Product
This dataset contains wave parameters derived from ENVISAT ASAR wave mode L1B product using CWAVE_ENV method, the retrived parameters are calibrated according to buoy data.


['Conventions',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__orthogonal_indexing__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_close',
 '_close_mem',
 '_enddef',
 '_getname',
 '_grpid',
 '_isopen',
 '_ncstring_attrs__',
 '_redef',
 'acq_station_name',
 'close',
 'cmptypes',
 'contact',
 'createCompoundType',
 'createDimension',
 'createEnumType',
 'createGroup',
 'createVLType',
 'createVariable',
 'cycle_number',
 'data_model',
 'delncattr',
 'dimensions',
 'disk_format',
 'end_date',
 'enumtypes',
 'file_format',
 'filepath',
 'fromcdl',
 'get_variables_by_attributes',
 'getncattr',
 'groups',
 'history',
 'imaging_mode',
 'isopen',
 'keepweakref',
 'mission_name',
 'name',
 'ncattrs'